## Imports

In [1]:
# Canonical
import pandas as pd

# HTTP
import requests

# File utilities
import gzip
import shutil
from pathlib import Path
import csv

# Math
import math

# Pretty print
from tabulate import tabulate

# Download raw data

In [4]:
raw_data_dir = Path("../data/raw")
raw_data_dir.mkdir(parents=True, exist_ok=True)

if any(raw_data_dir.iterdir()):
    pass

else:

    url = "https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv.gz"
    gz_file_path = raw_data_dir / "openfoodfacts_products.csv.gz"
    csv_file_path = raw_data_dir / "openfoodfacts_products.csv"

    print("Downloading OpenFoodFacts CSV file...")
    response = requests.get(url, stream=True)
    response.raise_for_status()

    with open(gz_file_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    print("Download complete. Extracting...")

    with gzip.open(gz_file_path, "rb") as f_in:
        with open(csv_file_path, "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)

    gz_file_path.unlink()

    print(f"Extraction complete: {csv_file_path}")

## Flag to avoid preprocessing twice

In [5]:
processed_data_dir = Path("../data/processed")
processed_file = processed_data_dir / "openfoodfacts_processed.csv"

processed_data_dir.mkdir(parents=True, exist_ok=True)

if any(processed_data_dir.iterdir()):
    df = pd.read_csv(processed_file)
    preprocessing = False

else:
    preprocessing = True

# Exploration

## Explore headings

In [10]:
if preprocessing:

    csv_file_path = raw_data_dir / "openfoodfacts_products.csv"

    with open(csv_file_path, "r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter="\t")
        headings = next(reader)
        num_cols = 8
        rows = [headings[i:i+num_cols] for i in range(0, len(headings), num_cols)]
        print(tabulate(rows, tablefmt="plain"))

code                   url                                      creator                            created_t                             created_datetime                                       last_modified_t                   last_modified_datetime     last_modified_by
last_updated_t         last_updated_datetime                    product_name                       abbreviated_product_name              generic_name                                           quantity                          packaging                  packaging_tags
packaging_en           packaging_text                           brands                             brands_tags                           categories                                             categories_tags                   categories_en              origins
origins_tags           origins_en                               manufacturing_places               manufacturing_places_tags             labels                                                 labels_ta

### Explore nulls

In [11]:
if preprocessing:

    chunk_size = 50_000  # Adjust as needed

    # Initialize a dictionary to track null counts
    null_counts = {}
    total_rows = 0

    for chunk in pd.read_csv(csv_file_path, sep="\t", dtype=str, chunksize=chunk_size, on_bad_lines="skip"):
        # Line 1663659 is bad

        total_rows += len(chunk)  # Track total rows processed

        # Count NaNs in this chunk
        chunk_null_counts = chunk.isna().sum()

        # Add to overall counts
        for col in chunk.columns:
            null_counts[col] = null_counts.get(col, 0) + chunk_null_counts[col]

    null_percentages = {col: (null_counts[col] / total_rows) * 100 for col in null_counts}

    null_df = pd.DataFrame.from_dict(null_percentages, orient="index", columns=["% Nulls"])

    # Show results
    pd.set_option("display.max_rows", None)  # Show all rows
    display(null_df)

,% Nulls
code,0.000000
url,0.000000
creator,0.000355
created_t,0.000000
created_datetime,0.000000
last_modified_t,0.000000
last_modified_datetime,0.000000
last_modified_by,3.290810
last_updated_t,0.009649
last_updated_datetime,0.009649


In the future, I'll make a way to see which chunk is being processed, as it takes too long

The vast, vast majority of columns have almost no data at all.

In [11]:
# Get the number of total rows so that I can show which chunk is being processed
if preprocessing:

    with open(csv_file_path, "r", encoding="utf-8") as f_in:
        total_lines = sum(1 for _ in f_in)

In [13]:
# Save only those columns that have > 50% non-null values

if preprocessing:

    # Filter the columns based on the '% Nulls' values
    columns_to_keep = null_df[null_df['% Nulls'] <= 50].index.tolist()

    total_chunks = math.ceil(total_lines / chunk_size)

    filtered_chunks = []

    for i, chunk in enumerate(pd.read_csv(csv_file_path, sep="\t", usecols=columns_to_keep,
                                          dtype=str, chunksize=chunk_size, on_bad_lines="skip")):

        print(f"\rProcessing chunk {i + 1}/{total_chunks}", end="", flush=True)

        filtered_chunks.append(chunk)

    print("\nChunks processed. Concatenating...")

    df = pd.concat(filtered_chunks, ignore_index=True)

    pd.set_option("display.max_columns", None)  # Show all columns
    display(df.head())

Processing chunk 74/74
Chunks processed. Concatenating...


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,last_updated_t,last_updated_datetime,product_name,brands,brands_tags,countries,countries_tags,countries_en,nutriscore_grade,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,completeness,last_image_t,last_image_datetime,image_url,image_small_url,image_nutrition_url,image_nutrition_small_url,energy-kcal_100g,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,sodium_100g
0,000000000054,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1582569031,2020-02-24T18:30:31Z,1733085204,2024-12-01T20:33:24Z,NaN,1738849326,2025-02-06T13:42:06Z,Limonade artisanale a la rose,NaN,NaN,en:fr,en:france,France,unknown,unknown,unknown,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",0.1625,1733085204,2024-12-01T20:33:24Z,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000000000063,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1673620307,2023-01-13T14:31:47Z,1732913331,2024-11-29T20:48:51Z,insectproductadd,1738849275,2025-02-06T13:41:15Z,Tablette Tanzanie,NaN,NaN,en:fr,en:france,France,unknown,unknown,unknown,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",0.2625,1732913326,2024-11-29T20:48:46Z,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000000000114,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1580066482,2020-01-26T19:21:22Z,1737247862,2025-01-19T00:51:02Z,smoothie-app,1738687801,2025-02-04T16:50:01Z,Chocolate n 3,Jeff de Bruges,jeff-de-bruges,France,en:france,France,unknown,unknown,unknown,"en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",0.475,1737247860,2025-01-19T00:51:00Z,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,NaN,2415,44,28,30,27,7.1,0.025,0.01
3,00000001,http://world-en.openfoodfacts.org/product/0000...,inf,1634745456,2021-10-20T15:57:36Z,1738676541,2025-02-04T13:42:21Z,waistline-app,1738848139,2025-02-06T13:22:19Z,KOJI MISO PASTE,UMAMI,umami,"Allemagne, États-Unis, en:fr","en:france,en:germany,en:united-states","France,Germany,United States",e,unknown,unknown,"en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",0.8625,1738676520,2025-02-04T13:42:00Z,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,209,874,6,1,21.4,11.1,11.2,11.8,4.72
4,0000000105,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1572117743,2019-10-26T19:22:23Z,1738073570,2025-01-28T14:12:50Z,NaN,1738850683,2025-02-06T14:04:43Z,Paleta gran reserva - Sierra nevada-,AdvoCare,advocare,"Spanien, Germany","en:germany,en:spain","Germany,Spain",unknown,Beverages,Artificially sweetened beverages,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",0.675,1738073557,2025-01-28T14:12:37Z,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


So only the macrunutrients remain. Another approach would be to select those rows that contain a sufficient percentage of data, or to build a model that is dynamic and makes predictions based on what data is present and not expecting all columns to be available.

Here is what happens if we select only columns with 10% of data but rows that have at least 90% of those columns:

In [14]:
if preprocessing:

    row_percentage = 0.9  # Proportion of non-NaN values required to keep a row

    filtered_data = pd.DataFrame()

    columns_to_keep = null_df[null_df['% Nulls'] <= 90].index.tolist()

    for i, chunk in enumerate(pd.read_csv(csv_file_path, sep="\t",
                                          usecols=columns_to_keep, dtype=str,
                                          chunksize=chunk_size,
                                          on_bad_lines="skip")):

        print(f"\rProcessing chunk {i + 1}/{total_chunks}", end="", flush=True)

        # Drop rows with < 50% non-NaN values
        chunk = chunk.dropna(thresh=int(len(chunk.columns) * row_percentage))


    print("\nChunks processed. Concatenating...")

    filtered_data = pd.concat([filtered_data, chunk], ignore_index=True)

    display(filtered_data.head())

Processing chunk 74/74
Chunks processed. Concatenating...


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,last_updated_t,last_updated_datetime,product_name,quantity,packaging,packaging_tags,packaging_en,brands,brands_tags,categories,categories_tags,categories_en,labels,labels_tags,labels_en,stores,countries,countries_tags,countries_en,ingredients_text,ingredients_tags,ingredients_analysis_tags,serving_size,serving_quantity,additives_n,additives_tags,additives_en,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,food_groups,food_groups_tags,food_groups_en,states,states_tags,states_en,environmental_score_score,environmental_score_grade,nutrient_levels_tags,product_quantity,unique_scans_n,popularity_tags,completeness,last_image_t,last_image_datetime,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kcal_100g,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,nutrition-score-fr_100g
0,9600340270593,http://world-en.openfoodfacts.org/product/9600...,sil,1565991524,2019-08-16T21:38:44Z,1644580311,2022-02-11T11:51:51Z,packbot,1737510954,2025-01-22T01:55:54Z,Tremoço extra,800 g,"Plástico, Lata","en:plastic,en:can","Plastic,Can",Azeifruto,azeifruto,"Alimentos e bebidas à base de plantas, Bebidas...","en:plant-based-foods-and-beverages,en:beverage...","Plant-based foods and beverages,Beverages,Plan...",de:Cal. 13/15,de:cal-13-15,de:cal-13-15,Aveirense,Deutschland,en:germany,Germany,"Tremoços, água, sal, regulador de acidez: E-27...","en:lupin-bean,en:vegetable,en:legume,en:pulse,...","en:palm-oil-free,en:vegan,en:vegetarian",15 g,15,3,"en:e211,en:e270,en:e330","E211 - Sodium benzoate,E270 - Lactic acid,E330...",-11,b,3,Beverages,Plant-based milk substitutes,en:plant-based-milk-substitutes,"en:beverages,en:plant-based-milk-substitutes","Beverages,Plant-based milk substitutes","en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,unknown,"en:fat-in-moderate-quantity,en:saturated-fat-i...",800,1,"bottom-25-percent-scans-2019,bottom-20-percent...",0.8875,1565991609,2019-08-16T21:40:09Z,de:tremoços,de:tremoços,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,NaN,1240,2.33,0.4,55.4,0.2,10,8.4,0.667,0.267,73.666,-11
1,96044353,http://world-en.openfoodfacts.org/product/9604...,openfoodfacts-contributors,1562003183,2019-07-01T17:46:23Z,1701195287,2023-11-28T18:14:47Z,samanthalandrews,1736283935,2025-01-07T21:05:35Z,NaN,120 g,NaN,NaN,NaN,crosta & mollica,crosta-mollica,"Aliments et boissons à base de végétaux, Alime...","en:plant-based-foods-and-beverages,en:plant-ba...","Plant-based foods and beverages,Plant-based fo...",Fabriqué en Italie,en:made-in-italy,Made in Italy,noze,France,en:france,France,"Farine de blé, fromage Parmesan 7% (lait), hui...","en:wheat-flour,en:cereal,en:flour,en:wheat,en:...","en:palm-oil-free,en:vegan-status-unknown,en:ve...",120g,120,0,NaN,NaN,20,e,3,Cereals and potatoes,Bread,en:bread,"en:cereals-and-potatoes,en:bread","Cereals and potatoes,Bread","en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",89,a,"en:fat-in-moderate-quantity,en:saturated-fat-i...",120,2,"bottom-25-percent-scans-2019,bottom-20-percent...",0.65,1569760741,2019-09-29T12:39:01Z,en:breadsticks,Breadsticks,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,h

In [15]:
if preprocessing:

    print(filtered_data.shape)

(76, 75)


So we still have only macrunutrients and now not enough rows. As a final option, I'll select the top 10,000, 1,000, and 200 rows based on completeness of data, remove the columns that are still causing problems, and then remove the rows that are incomplete. I will also get rid of the useless columns based on discussion with you.

In [60]:
def select_top_n_rows_by_non_nulls(n, csv_file_path, chunk_size=50_000):

    df = pd.DataFrame()

    for i, chunk in enumerate(pd.read_csv(csv_file_path, sep="\t", dtype=str,
                                          chunksize=chunk_size,
                                          on_bad_lines="skip")):

        print(f"\rProcessing chunk {i + 1}/{total_chunks}", end="", flush=True)

        chunk['non_null_count'] = chunk.notna().sum(axis=1)

        chunk_sorted = chunk.sort_values(by='non_null_count', ascending=False)

        chunk_sorted = chunk_sorted.drop(columns=['non_null_count'])

        df = pd.concat([df, chunk_sorted.head(n)], ignore_index=True)

        df['non_null_count'] = df.notna().sum(axis=1)

        df = df.sort_values(by='non_null_count', ascending=False)

        df = df.drop(columns=['non_null_count'])

        df = df.head(n)

    return df

def select_columns_by_non_null(df, threshold=0.7):

    non_null_counts = df.notna().sum()

    selected_columns = non_null_counts[non_null_counts >= threshold * len(df)].index

    return df[selected_columns]

In [ ]:
if preprocessing:

    print("Processing top 200 rows with most non-null values...")
    df_200 = select_top_n_rows_by_non_nulls(200, csv_file_path)

    print("\nProcessing top 1000 rows with most non-null values...")
    df_1000 = select_top_n_rows_by_non_nulls(1000, csv_file_path)

    print("\nProcessing top 10,000 rows with most non-null values...")
    df_10_000 = select_top_n_rows_by_non_nulls(10_000, csv_file_path)

Processing chunk 74/74

Let's keep the columns that have > 70% data

In [ ]:
if preprocessing:

    df_200 = select_columns_by_non_null(df_200)
    df_1000 = select_columns_by_non_null(df_1000)
    df_10_000 = select_columns_by_non_null(df_10_000)

In [ ]:
if preprocessing:

    missing_percent = (df_1000.isna().sum() / len(df_1000)) * 100
    print(missing_percent)

code                                                     0.0
url                                                      0.0
creator                                                  0.0
created_t                                                0.0
created_datetime                                         0.0
last_modified_t                                          0.0
last_modified_datetime                                   0.0
last_modified_by                                         0.4
last_updated_t                                           0.0
last_updated_datetime                                    0.0
product_name                                             0.4
quantity                                                 0.8
brands                                                   0.1
brands_tags                                              0.1
categories                                               0.0
categories_tags                                          0.0
categories_en           

In [ ]:
# Let's be more aggressive and drop columns with > 10% missing values

if preprocessing:

    df_200 = select_columns_by_non_null(df_200, 0.9)
    df_1000 = select_columns_by_non_null(df_1000, 0.9)
    df_10_000 = select_columns_by_non_null(df_10_000, 0.9)

In [ ]:
if preprocessing:

    missing_percent = ((df_1000.isna().sum() / len(df_1000)) * 100).sort_values(ascending=False)
    print(missing_percent)

image_ingredients_small_url                              9.1
image_ingredients_url                                    9.1
image_nutrition_url                                      8.3
image_nutrition_small_url                                8.3
popularity_tags                                          7.1
unique_scans_n                                           6.7
fiber_100g                                               6.3
energy-kcal_100g                                         5.4
image_small_url                                          1.5
image_url                                                1.5
nova_group                                               1.4
sugars_100g                                              1.1
saturated-fat_100g                                       0.9
quantity                                                 0.8
product_quantity                                         0.8
product_name                                             0.4
last_modified_by        

In [ ]:
# We don't need those first six
if preprocessing:

    df_200 = select_columns_by_non_null(df_200, 0.935)
    df_1000 = select_columns_by_non_null(df_1000, 0.935)
    df_10_000 = select_columns_by_non_null(df_10_000, 0.935)

In [ ]:
# Let's try to remove rows with more than 10% missing values
if preprocessing:

    df_200 = df_200.dropna(thresh=int(0.9 * df_200.shape[1]))
    df_1000 = df_1000.dropna(thresh=int(0.9 * df_1000.shape[1]))
    df_10_000 = df_10_000.dropna(thresh=int(0.9 * df_10_000.shape[1]))

    print(df_200.shape, df_1000.shape, df_10_000.shape)

(197, 65) (999, 50) (9826, 64)


In [ ]:
# Let's try 95% now
if preprocessing:

    df_200 = df_200.dropna(thresh=int(0.95 * df_200.shape[1]))
    df_1000 = df_1000.dropna(thresh=int(0.95 * df_1000.shape[1]))
    df_10_000 = df_10_000.dropna(thresh=int(0.95 * df_10_000.shape[1]))

    print(df_200.shape, df_1000.shape, df_10_000.shape)

(192, 65) (993, 50) (9491, 64)


In [ ]:
if preprocessing:

    print(((df_200.isna().sum() / len(df_200)) * 100).sort_values(ascending=False))

image_ingredients_url                                    4.687500
image_ingredients_small_url                              4.687500
unique_scans_n                                           4.166667
phosphorus_100g                                          4.166667
popularity_tags                                          4.166667
energy-kcal_100g                                         3.645833
image_nutrition_url                                      3.125000
image_nutrition_small_url                                3.125000
magnesium_100g                                           2.604167
vitamin-b6_100g                                          2.604167
vitamin-a_100g                                           2.604167
sugars_100g                                              2.604167
nova_group                                               2.083333
saturated-fat_100g                                       1.562500
salt_100g                                                1.562500
sodium_100

In [ ]:
if preprocessing:

    print(((df_1000.isna().sum() / len(df_1000)) * 100).sort_values(ascending=False))

fiber_100g                                               6.042296
energy-kcal_100g                                         5.135952
nova_group                                               1.208459
image_small_url                                          1.208459
image_url                                                1.208459
sugars_100g                                              0.906344
product_quantity                                         0.704935
saturated-fat_100g                                       0.704935
quantity                                                 0.704935
product_name                                             0.402820
salt_100g                                                0.402820
sodium_100g                                              0.402820
last_modified_by                                         0.302115
countries                                                0.201410
countries_tags                                           0.201410
countries_

In [ ]:
if preprocessing:

    print(((df_10_000.isna().sum() / len(df_10_000)) * 100).sort_values(ascending=False))

energy-kcal_100g                                         4.983669
labels                                                   3.613950
labels_en                                                3.613950
labels_tags                                              3.613950
popularity_tags                                          2.497103
nutrition-score-fr_100g                                  2.233695
nutriscore_score                                         2.233695
image_ingredients_small_url                              1.864925
image_ingredients_url                                    1.864925
unique_scans_n                                           1.801707
image_nutrition_small_url                                1.506691
image_nutrition_url                                      1.506691
food_groups_en                                           1.348646
food_groups_tags                                         1.348646
food_groups                                              1.348646
nova_group

In [ ]:
# Let's try 100%
if preprocessing:

    df_200 = df_200.dropna()
    df_1000 = df_1000.dropna()
    df_10_000 = df_10_000.dropna()

    print(df_200.shape, df_1000.shape, df_10_000.shape)

(130, 65) (841, 50) (7793, 64)


In [ ]:
if preprocessing:

    display(df_1000.columns)

Index(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'last_modified_by',
       'last_updated_t', 'last_updated_datetime', 'product_name', 'quantity',
       'brands', 'brands_tags', 'categories', 'categories_tags',
       'categories_en', 'countries', 'countries_tags', 'countries_en',
       'ingredients_text', 'ingredients_tags', 'ingredients_analysis_tags',
       'additives_n', 'nutriscore_grade', 'nova_group', 'pnns_groups_1',
       'pnns_groups_2', 'states', 'states_tags', 'states_en',
       'product_quantity', 'completeness', 'last_image_t',
       'last_image_datetime', 'main_category', 'main_category_en', 'image_url',
       'image_small_url', 'energy-kcal_100g', 'energy_100g', 'fat_100g',
       'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g',
       'proteins_100g', 'salt_100g', 'sodium_100g',
       'fruits-vegetables-nuts-estimate-from-ingredients_100g'],
      dtype='object')

In [ ]:
if preprocessing:

    df = df_1000[['code', 'url', 'product_name', 'quantity', 'brands', 'brands_tags',
                  'categories', 'categories_tags', 'categories_en', 'countries',
                  'countries_tags', 'countries_en', 'ingredients_text',
                  'ingredients_tags', 'ingredients_analysis_tags', 'additives_n',
                  'product_quantity', 'main_category', 'main_category_en',
                  'image_url', 'image_small_url', 'energy-kcal_100g', 'energy_100g',
                  'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g',
                  'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g',
                  'fruits-vegetables-nuts-estimate-from-ingredients_100g',
                  'nutriscore_grade', 'nova_group']]

In [ ]:
if preprocessing:

    df.to_csv(processed_file, index=False, quoting=csv.QUOTE_ALL)